In [3]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy as sp
from scipy import misc
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from nltk.corpus import stopwords

import sklearn as skl
import csv
import re
import json
import string

#### read in data

In [4]:
file_train = "CS5785-final-data/train.txt"
path_to_images = "CS5785-final-data/images/test/"

In [5]:
#all images are 256x 256 x 3
im = misc.imread(path_to_images + "0001.jpg")
im.shape

(256, 256, 3)

In [116]:
#use alex net data
alex_net_feat_10k = np.load("CS5785-final-data/alexnet_feat_10k.npy")
alex_net_feat_test = np.load("CS5785-final-data/alexnet_feat_test.npy")
alex_net_feat_train = np.load("CS5785-final-data/alexnet_feat_train.npy")
#sift data
sift_feat_10k = np.load("CS5785-final-data/SIFTBoW_10k.npy")
sift_feat_test = np.load("CS5785-final-data/SIFTBoW_test.npy")
sift_feat_train = np.load("CS5785-final-data/SIFTBoW_train.npy")

In [6]:
#read in attributes
file_a = "CS5785-final-data/attributes_list.txt"
file_b = "CS5785-final-data/attributes_test.txt"
file_c = "CS5785-final-data/attributes_train.txt"
file_out = "CS5785-final-data/train.txt"

In [7]:
with open(file_a, 'r') as f:
    reader = csv.reader(f, dialect='excel', delimiter='\n')
    attributes_list = []
    for row in reader: 
        attributes_list.append(row[0])

In [8]:
def read_att(file):
    with open(file, 'r') as f:
        reader = csv.reader(f, dialect='excel', delimiter='\n')
        img = []
        fv = []
        for row in reader: 
            img.append(row[0].split()[0])
            fv.append(map(int, row[0].split()[1].split(",")))
    return img, fv

In [9]:
img, fv = read_att(file_c)
final_test_img, final_test_fv = read_att(file_b)

In [10]:
with open(file_out, 'r') as f:
    reader = csv.reader(f, dialect='excel', delimiter='\n')
    output = []
    for row in reader: 
        output.append(row[0].split()[1])


In [11]:
len(output)

3000

### Load captions

In [95]:
f ="CS5785-final-data/captions.json"

with open(f) as json_file:
    j = json.load(json_file)

In [176]:
# convert to string
stop_words = set(stopwords.words("english"))

captions = []
imgs = []
for caption in j:
    words = []
    for pic in j[caption]:
        caption_text = pic.encode('ascii','ignore')
        caption_text = caption_text.translate(None, string.punctuation)
        caption_text = caption_text.split()
        caption_text = [c.lower() for c in caption_text]
        caption_text = [c for c in caption_text if c not in stop_words]
        
        words.extend(caption_text)
    captions.append(words)
    imgs.append(caption)

In [ ]:
for r in xrange(len(output)):
    output[r] = output[r].replace("_", " ")

In [52]:
unique_lab = np.unique(np.asarray(output))
# Clean some words
unique_lab = [u.replace('videostore','video store') for u in unique_lab]

##Google news word2vec model from:

https://code.google.com/p/word2vec/

In [22]:
# from gensim.models import Word2Vec
import os
import gensim
model = gensim.models.word2vec.Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
model.similarity('woman', 'man')


0.76640122344103201

In [264]:
model.similarity('playground', 'food')


0.087995496112116989

In [107]:
print captions[0]
print output[0]
c = [model[word] for word in captions[0]]
o = [model[word] for word in unique_lab[0].split(' ')]

['crowds', 'cheer', 'cyclists', 'complete', 'race', 'finish', 'bicycle', 'road', 'racers', 'passing', 'group', 'cheering', 'spectators', 'people', 'bicycles', 'riding', 'group', 'onlookers', 'bicycle', 'racers', 'cheered', 'people', 'racers', 'bikes', 'riding', 'past', 'crowd']
yard


In [149]:
from gensim import matutils

def n_similarity(model, ws1, ws2):
    v1 = []
    v2 = []
    bad_words = []
    for word in ws1:
        try: 
            v1.append(model[word])
        except:
            bad_words.append(word)
            continue
    for word in ws2:
        try: 
            v2.append(model[word])
        except:
            bad_words.append(word)
            continue
    return np.dot(matutils.unitvec(np.array(v1).mean(axis=0)), matutils.unitvec(np.array(v2).mean(axis=0))), bad_words

In [162]:
gensim_features = np.zeros((10000, 200))
list_of_bad_words = []
for u_idx, u in enumerate(unique_lab):
    u = u.split(' ')
    for c_idx, c in enumerate(captions):
        gensim_features[c_idx, u_idx], _ = n_similarity(model, c, u)
        # gensim_features[c_idx, u_idx], bad_words = n_similarity(model, c, u)
        # list_of_bad_words.extend(bad_words)


In [249]:
np.save("gensim_features.npy", gensim_features)
np.save("imgs.npy", imgs)
np.save("captions.npy", captions)
np.save("unique_lab.npy", unique_lab)
np.save("output.npy", output)

In [239]:
idx = 18
print np.argmax(gensim_features[idx])
print np.max(gensim_features[idx])
print unique_lab[np.argmax(gensim_features[idx])]
print imgs[idx]
print captions[idx]

171
0.368644805917
soccer field
2587106431.jpg
['three', 'young', 'boys', 'dressed', 'orange', 'navy', 'shirt', 'smiling', 'signs', 'fingers', 'three', 'boys', 'dressed', 'blue', 'orange', 'smiling', 'using', 'hand', 'signals', 'three', 'young', 'boys', 'wearing', 'school', 'uniforms', 'pose', 'smile', 'camera', 'three', 'young', 'boys', 'flash', 'signs', 'hands', 'smile', 'three', 'children', 'laughing', 'playing', 'together']


In [222]:
gmax = [np.max(g) for g in gensim_features]


In [160]:
all_caption_words = set()
for w in captions:
    for ww in w:
        all_caption_words.add(ww)

In [158]:
len(set(list_of_bad_words))


1081

In [190]:
x = 0
fv_10k = []
for label in output:
    x+=1
    if label in captions[0]:
         fv_10k.append(1)
    else:
        fv_10k.append(0)
print x

3000


In [133]:
j['3304712466.jpg'][0].encode('ascii','ignore').split()
a = []
for r in j['3304712466.jpg']:
    a.extend(r.encode('ascii','ignore').split())